In [1]:
import time

import numpy as np
import pandas as pd
from scipy.sparse import rand, csr_matrix

## Problem:


#### R $\in \mathbb{R}^{n*m}$ - feedback Matrix (explicit/implicit);
 
#### S $\in$ $\mathbb{R}^{m*m}$ - similarity matrix of items

#### $\Rightarrow$ RS = $\Lambda$R, $\Lambda$ - diagonal matrix, with "eigenvalues" on diagonal;

#### Find "S" s.t.:
##### 1. Not identical
##### 2. $\forall$ row r$_{u}$ $\in$ R is a left "eigenvector" of S

## Optimization problem:

##### Let $\Lambda_{ii} = \lambda_{i}$,
$$\mathcal{L}_{k}(S, \Lambda) = \frac{1}{2} \|\Lambda R - RS \|^{2}_{F} + \frac{\alpha}{2}\|S \|^{2}_{F}
+ \beta \|S \|_{1} + \frac{\gamma}{2}\|\Lambda \|^{2}_{F} + k \sum_{i=1}^m(max(0, \varepsilon - \lambda_{i}))^{2}
\rightarrow \min: s.t. S_{ij} \ge0 , diag(S) = 0
$$

### Step 1:

#### Fixed $\Lambda$, search for S with the formula $\forall$ p, q = 1 $\cdots$ n:

$$ S_{pq} =
\begin{cases}
  \dfrac{\rho_{pq} + \beta}{z_{p}} , & \rho_{pq} < -\beta\\      
  0 , & \lvert \rho_{pq} \rvert < \beta\\
  \dfrac{\rho_{pq} - \beta}{z_{p}} , & \rho_{pq} > \beta \\
\end{cases}
$$

$$
 where ~ \rho_{pq} = \sum_{i=1}^m r_{ip}(\lambda_{i}r_{iq} - \sum_{k=1, k\neq p}^n r_{ik}S_{kq}), ~
z_{p} = \beta + \sum_{i=1}^m r_{ip}^{2}
$$


### Step 2:

#### Fixed S, search for $\Lambda$ iteratively with the formula $\forall$ p = 1 $\cdots$ m, k = 10 $\cdots$ :

$$
\lambda_{p}^{n+1} = \lambda_{p}^{n} - \kappa_{n} \frac{\partial \mathcal{L}_{k}}{\partial \lambda_{p}} ~ 	
$$

$$
where ~ \frac{\partial \mathcal{L}_{k}}{\partial \lambda_{p}} = \lambda_{p} y_{p} - \rho_{p} - 2k *
max(0, \varepsilon - \lambda_{i}), ~ y_{p} = \gamma + \sum_{j=1}^n r_{pj}^{2}, ~
\rho_{p} = \sum_{j=1}^n r_{pj} \sum_{k=1}^n r_{pk} S_{kj}
$$

### And back and forth Step 1, 2 until little change of error.

### Important:
#### $\Lambda$ and S are filled with random numbers initially.
#### $\gamma$ = 0, to try to prevent $\lambda_{i} \rightarrow$ 0
#### The last term in the optimization problem stands for penalty for every $\lambda_{i} \leq 0$, we want $\lambda_{i} > 0$, but alternatively $\lambda_{i} \geq \varepsilon$
#### $\kappa_{n}  \rightarrow 0, n  \rightarrow \infty$
#### $k \rightarrow \infty$ - parameter in Penalty functions optimization

## Class to solve EigenSimilarity problem:

In [2]:
class EigenSimModel:
    def __init__(self, feedback, alpha=0.05,
                 beta=0.05, gamma=0.0,
                 kappa=0.05, e_lambda = 1,
                 random_state=1, mu=2,
                 std=0.5, num_iter=5, batch_size = 5,
                 num_iter_l=5, eps=0.01):
        
        
        self.alp = alpha 
        self.beta = beta
        self.gamma = gamma
        self.kappa = kappa
        self.e_lambda = e_lambda # epsilon for penalty
        self.k = 10 # parameter for penalty function
        self.mu = mu
        self.std = std
        self.num_iter = num_iter # global iterations
        self.num_iter_l = num_iter_l # local lambda iterations
        self.eps = eps # change of error we want to achieve
        self.feedback = feedback # MUST BE SPARSE CSR MATRIX
        self.batch_size = batch_size
        
        self.m = feedback.shape[0] # num of users
        self.n = feedback.shape[1] # num of items
        self.lamb = self.std * np.random.rand(self.m) + self.mu
        self.model = np.random.rand(self.n, self.n)
        self.model -= np.diag(np.diag(self.model)) # diagonal is zero
        
        # denominator to find model entities
        self._z_model = np.sum(feedback.toarray()**2, axis=0) + self.beta
        
        # denominator to find lambda entities
        self._z_lamb = np.sum(feedback.toarray()**2, axis=1) + self.gamma
        
        
    # Update model S:
    def _update_model_pq(self, p, q, feedback):
        sq = self.model[:, q]
        sq[p] = 0.0
        rpq = np.sum(feedback[:, p] 
                     * (self.lamb * feedback[:, q] - self.feedback.dot(sq)))
        if np.abs(rpq) < self.beta:
            return 0.0
        elif rpq < -self.beta:
            return 0.0 # to return only positive numbers to fit constraints
            #return (rpq + self.beta) / self._z_model[p]
        elif rpq > self.beta:
            return (rpq - self.beta) / self._z_model[p]
        
        # for problems
        assert False 
    
    
    def _update_model(self, feedback):
        p_set = np.random.randint(self.n, size=self.batch_size)
        q_set = np.random.randint(self.n, size=self.batch_size)
        for p in p_set:#range(self.n):
            for q in q_set:#range(self.n):
                if p != q:
                    self.model[p, q] = self._update_model_pq(
                        p,
                        q,
                        feedback
                    )
              
    
    # Update Lambda:
    def _update_lambda_p(self, p, feedback):
        rp = np.sum(feedback[p, :] 
                    * feedback[p, :].dot(self.model))
        result = (self.lamb[p] * self._z_lamb[p] - rp
                  - 2 * self.k * np.max((0, self.e_lambda - self.lamb[p])))
        return result 
    
    def _update_lambda(self, feedback):
        former_err = 0.0
        latter_err = self._get_error(feedback)
        n_iter = 0
        while np.abs(former_err - latter_err) > self.eps:
            for p in range(self.m):
                self.lamb[p] -= self.kappa * self._update_lambda_p(p, feedback)
            
            former_err = latter_err
            latter_err = self._get_error(feedback)
            n_iter += 1
            if n_iter == self.num_iter_l:
                break
             
    # Find error on matrices:
    def _get_error(self, feedback):
        res = (self.lamb.reshape((self.m, 1)) 
               * feedback - self.feedback.dot(self.model))
        first = 0.5 * np.linalg.norm(res)**2
        second = (self.alp / 2) * np.linalg.norm(self.model)**2
        third = self.beta * np.sum(np.abs(self.model))
        forth = (self.gamma / 2) * np.linalg.norm(self.lamb)**2
        return first + second + third + forth
    
    
    def fit(self, feedback, show_step=(True, 10), k=20):
        # feedback MUST BE NOT A SPARSE MATRIX
        self.k = k
        former_err = 0.0
        latter_err = self._get_error(feedback)
        n_iter = 0
        print("Begin calculations:\n")
        start = time.time()
        
        while np.abs(former_err - latter_err) > self.eps:
            # evaluate entries of model matrix
            self._update_model(feedback)
            
            # evaluate entries of lambda matrix
            self._update_lambda(feedback)
            
            former_err = latter_err
            latter_err = self._get_error(feedback)
            
            n_iter += 1
            if show_step[0]:
                if n_iter % show_step[1] == 0:
                    end = time.time()
                    print(
                        f"Iteration: {n_iter}, "
                        + f"Error: {latter_err}, " 
                        + f"Time for {show_step[1]} steps: {end - start}\n\n"
                    )
                    start = time.time()

            if n_iter == self.num_iter:
                self.get_statistics(feedback)
                print("Ended with iterations\n\n")
                break
                
        #print(f"Got error = {latter_err};\n")       
    
    
    def get_model(self):
        return self.lamb, self.model
    
    
    def get_statistics(self, data):
        merr = np.linalg.norm(self.lamb.reshape((data.shape[0], 1)) 
               * data - data.dot(self.model)) / np.multiply(*data.shape)
        lamb_norm = np.linalg.norm(self.lamb)
        model_norm = np.linalg.norm(self.model)
        density = (self.model>0).sum() / np.multiply(*self.model.shape)
        lmax = self.lamb.max()
        lmin = self.lamb.min()
        lmean = self.lamb.mean()
        
        
        print(f"#############################\nPenalty parameter 'K' = {self.k}\n"
              + f"\nMSE = {merr}\n\n"
              + f"Model norm = {model_norm}\nModel density = {density}\n"
              + f"Number of elements less than 0 in Model = {(self.model<0).sum()}\n"
              + f"Maximal element of the Model = {self.model.max()}\n"
              + f"Minimal element of the Model = {self.model.min()}\n"
              + f"\nLambda norm = {lamb_norm}\nLambda max = {lmax}\n"
              + f"Lambda min = {lmin}\nLambda mean = {lmean}\n"
              + f"#############################")

In [3]:
# Model true data
m, n = 750, 350
random_state = 5
data = rand(m, n, density=0.016, format="csr", random_state=42)
data[data > 0] = 1.0

In [4]:
model = EigenSimModel(
    data,
    alpha=0.007,
    beta=0.6,
    gamma=0.0002,
    kappa=0.02,
    e_lambda = 0.3,
    random_state=2,
    mu=-1,
    std=0.5,
    num_iter=12,
    num_iter_l=500,
    eps=1e-3,
    batch_size=int(n/2)
)

In [5]:
# THIS IS USED THIS WAY TO ACCELERATE CALCULATIONS
data = data.toarray()

In [6]:
K = [100, 200, 250, 300, 400, 600, 800, 1000]

In [7]:
for k in K:
    model.fit(data, show_step=(True, 4), k=k)

Begin calculations:

Iteration: 4, Error: 462577.59780520556, Time for 4 steps: 48.92401146888733


Iteration: 8, Error: 180351.57149946713, Time for 4 steps: 40.68249559402466


Iteration: 12, Error: 73069.24581669018, Time for 4 steps: 104.6824221611023


#############################
Penalty parameter 'K' = 100

MSE = 0.001404412676239185

Model norm = 75.15412286764078
Model density = 0.14118367346938776
Number of elements less than 0 in Model = 0
Maximal element of the Model = 0.9999205866059392
Minimal element of the Model = 0.0

Lambda norm = 13.164622771308915
Lambda max = 1.7124369250723157
Lambda min = 0.2679927534024286
Lambda mean = 0.429901429228086
#############################
Ended with iterations


Begin calculations:

Iteration: 4, Error: 30894.189579487924, Time for 4 steps: 156.10768699645996


Iteration: 8, Error: 12487.273030779612, Time for 4 steps: 148.3799967765808


Iteration: 12, Error: 7033.261252303088, Time for 4 steps: 155.03919577598572


###############

# MISC

model = EigenSimModel(
    data,
    alpha=0.001,
    beta=0.3,
    gamma=0.0,
    kappa=0.02,
    e_lambda = 0.3,
    random_state=2,
    mu=5,
    std=0.9,
    num_iter=5,
    num_iter_l=3,
    eps=1e-3,
    batch_size=500
)

In [ ]:
p, q = (5, 6)
feedback = data.toarray()
model.batch_size = 60
model.num_iter_l = 1

In [ ]:
%%time
model._update_model_pq(p, q, feedback)

In [ ]:
%%time
model._update_model(feedback)

In [ ]:
%%time
model._update_lambda_p(p, feedback)

In [ ]:
%%time
model._update_lambda(feedback)

In [ ]:
%%time
model._get_error(feedback)